In [5]:
%reset -f
from tokenizers import Tokenizer, AddedToken
import json
from tokenizers.models import BPE
import gc
import torch
CUDA_LAUNCH_BLOCKING=1
gc.collect()
torch.cuda.empty_cache()
torch.manual_seed(42)
import random
random.seed(42)

In [6]:
import transformers
transformers.__version__

'4.34.1'

path = '../../protgpt2/'

tokenizer = Tokenizer.from_file(path+'tokenizer.json')
special_tokens = [
    AddedToken("<BOS>", single_word=True, normalized=True),
    AddedToken("<EOS>", single_word=True, normalized=True),
    AddedToken("<PAD>", single_word=True, normalized=True),
]

tokenizer.add_special_tokens(special_tokens)

In [7]:
from tokenizers import Tokenizer

def load_tokenizer_with_special_tokens(path):
    # Load the tokenizer
    tokenizer = Tokenizer.from_file(path + 'tokenizer.json')

    # Set bos_token_id and eos_token_id to 0
    tokenizer.bos_token_id = 0
    tokenizer.eos_token_id = 0

    return tokenizer

path = '../../protgpt2/'  # replace with your directory
tokenizer = load_tokenizer_with_special_tokens(path)

# Confirming
assert tokenizer.bos_token_id == 0
assert tokenizer.eos_token_id == 0


In [8]:
# initialize config for the model
from transformers.models.llama.configuration_llama import LlamaConfig
MODEL_CONFIGS = {
    'protllama2': LlamaConfig(max_position_embeddings=1024, # maximum length
                      hidden_size = 1280,
                      bos_token_id = tokenizer.bos_token_id,
                      eos_token_id = tokenizer.eos_token_id,
                      transformers_version = transformers.__version__,
                      intermediate_size = 3440,
                      vocab_size = 50257),
    '7b': LlamaConfig(max_position_embeddings=512,
                      hidden_size = 640,
                      intermediate_size = 1720,
                      _flash_attn_2_enabled=True)}

In [9]:
MODEL_CONFIGS

{'protllama2': LlamaConfig {
   "attention_bias": false,
   "bos_token_id": 0,
   "eos_token_id": 0,
   "hidden_act": "silu",
   "hidden_size": 1280,
   "initializer_range": 0.02,
   "intermediate_size": 3440,
   "max_position_embeddings": 1024,
   "model_type": "llama",
   "num_attention_heads": 32,
   "num_hidden_layers": 32,
   "num_key_value_heads": 32,
   "pretraining_tp": 1,
   "rms_norm_eps": 1e-06,
   "rope_scaling": null,
   "rope_theta": 10000.0,
   "tie_word_embeddings": false,
   "transformers_version": "4.34.1",
   "use_cache": true,
   "vocab_size": 50257
 },
 '7b': LlamaConfig {
   "attention_bias": false,
   "bos_token_id": 1,
   "eos_token_id": 2,
   "hidden_act": "silu",
   "hidden_size": 640,
   "initializer_range": 0.02,
   "intermediate_size": 1720,
   "max_position_embeddings": 512,
   "model_type": "llama",
   "num_attention_heads": 32,
   "num_hidden_layers": 32,
   "num_key_value_heads": 32,
   "pretraining_tp": 1,
   "rms_norm_eps": 1e-06,
   "rope_scaling": n

In [14]:
if not getattr(MODEL_CONFIGS['7b'], '_flash_attn_2_enabled', False)

SyntaxError: invalid syntax (397744783.py, line 1)

In [10]:
from flash_attn import flash_attn_func, flash_attn_varlen_func
from flash_attn.bert_padding import index_first_axis, pad_input, unpad_input  # noqa


In [11]:
if not getattr(MODEL_CONFIGS['7b'], "_flash_attn_2_enabled", False):
    # This block is executed if _flash_attn_2_enabled attribute is either missing or set to a falsy value
    print('yes')
else:
    # This block is executed if _flash_attn_2_enabled attribute is set to a truthy value
    print('wrong')


wrong


In [12]:
from transformers.models.llama.modeling_llama import LlamaForCausalLM
import flash_attn
model = LlamaForCausalLM(MODEL_CONFIGS['7b'])
print(model.config)  # If this attribute exists and is True, flash_attn is likely being used.
model

2023-10-19 20:38:52.304255: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


LlamaConfig {
  "attention_bias": false,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 640,
  "initializer_range": 0.02,
  "intermediate_size": 1720,
  "max_position_embeddings": 512,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "transformers_version": "4.34.1",
  "use_cache": true,
  "vocab_size": 32000
}



LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 640)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear(in_features=640, out_features=640, bias=False)
          (k_proj): Linear(in_features=640, out_features=640, bias=False)
          (v_proj): Linear(in_features=640, out_features=640, bias=False)
          (o_proj): Linear(in_features=640, out_features=640, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=640, out_features=1720, bias=False)
          (up_proj): Linear(in_features=640, out_features=1720, bias=False)
          (down_proj): Linear(in_features=1720, out_features=640, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Li

In [15]:
model = LlamaForCausalLM(MODEL_CONFIGS['7b'])
for layer in model.model.layers:
    if 'LlamaFlashAttention2' in str(type(layer.self_attn)):
        print("Using flash_attn in one of the decoder layers.")
        break


Using flash_attn in one of the decoder layers.


In [ ]:
if is_flash_attn_2_available():
    from flash_attn import flash_attn_func, flash_attn_varlen_func
    from flash_attn.bert_padding import index_first_axis, pad_input, unpad_input  # noqa


In [7]:
model = LlamaForCausalLM(MODEL_CONFIGS['7b'])
for layer in model.model.layers:
    if isinstance(layer.self_attn, LlamaFlashAttention2):
        print("Using flash_attn in one of the decoder layers.")
        break


NameError: name 'LlamaFlashAttention2' is not defined

In [1]:
from datasets import load_dataset
import pickle

In [2]:
with open('/data/rozen/home/e0833634/lama/protllama/original_lama.pkl', 'rb') as f:
    loaded_data_lama = pickle.load(f)
#name = 'togethercomputer/RedPajama-Data-1T-Sample'
loaded_data_lama

DatasetDict({
    train: Dataset({
        features: ['text', 'meta'],
        num_rows: 930514
    })
})

In [8]:
train_ds = loaded_data_lama['train']


In [9]:
from datasets import Dataset
valid_dict = {'text': ["Since it was initiated by the Brazil workers' party~\cite{wainwright2003making} in the 90s, Participatory budgeting (PB)~\cite{cabannes2004participatory}"]}
valid_dataset = Dataset.from_dict(valid_dict)

In [10]:
from functools import partial
from colossalai.nn.lr_scheduler import CosineAnnealingWarmupLR
from colossalai.nn.optimizer import HybridAdam
from torch.optim import Optimizer
from typing import Optional, Tuple
from transformers.models.llama.tokenization_llama import LlamaTokenizer

2023-09-21 14:55:21.409685: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
from data_utils_modified import prepare_dataloader_modified
tokenizer = LlamaTokenizer.from_pretrained('hf-internal-testing/llama-tokenizer')
# follows fast chat: https://github.com/lm-sys/FastChat/blob/main/fastchat/train/train.py#L257
tokenizer.pad_token = tokenizer.unk_token
def tokenize_batch(batch, tokenizer: Optional[LlamaTokenizer] = None, max_length: int = 1024):
    texts = [sample['text'] for sample in batch]
    data = tokenizer(texts, return_tensors="pt", padding='max_length', truncation=True, max_length=max_length)
    data['labels'] = data['input_ids'].clone()
    return data

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [12]:
dataloader = prepare_dataloader_modified(train_ds, batch_size=2, shuffle=False,
                                drop_last=True,
                                collate_fn=partial(tokenize_batch, tokenizer=tokenizer, max_length=1024))

In [18]:
valid_dataloader = prepare_dataloader_modified(valid_dataset, batch_size=2, shuffle=False,
                                drop_last=False,
                                collate_fn=partial(tokenize_batch, tokenizer=tokenizer, max_length=1024))

In [19]:
for step, batch in enumerate(valid_dataloader):
    for k, v in batch.items():
        print(k,v)

input_ids tensor([[   1, 4001,  372,  ...,    0,    0,    0]])
attention_mask tensor([[1, 1, 1,  ..., 0, 0, 0]])
labels tensor([[   1, 4001,  372,  ...,    0,    0,    0]])


In [20]:
start_epoch = 0
start_step = 0
sampler_start_idx = 0
num_steps_per_epoch = len(dataloader)
num_epoch = 4

In [21]:
num_steps_per_epoch

465257

In [54]:
from transformers.models.llama.modeling_llama import LlamaForCausalLM
torch.manual_seed(42)
import random
random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
config = '7b'
config = MODEL_CONFIGS[config]
model = LlamaForCausalLM(config)

In [55]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 640)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=640, out_features=640, bias=False)
          (k_proj): Linear(in_features=640, out_features=640, bias=False)
          (v_proj): Linear(in_features=640, out_features=640, bias=False)
          (o_proj): Linear(in_features=640, out_features=640, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=640, out_features=1720, bias=False)
          (up_proj): Linear(in_features=640, out_features=1720, bias=False)
          (down_proj): Linear(in_features=1720, out_features=640, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(i

In [56]:
model.lm_head.weight

Parameter containing:
tensor([[-0.0190, -0.0132,  0.0111,  ...,  0.0009,  0.0142, -0.0255],
        [-0.0295,  0.0292,  0.0068,  ..., -0.0125,  0.0372, -0.0088],
        [-0.0024, -0.0122, -0.0183,  ...,  0.0047,  0.0111, -0.0331],
        ...,
        [ 0.0007, -0.0253, -0.0171,  ..., -0.0376,  0.0025,  0.0206],
        [-0.0065,  0.0047, -0.0297,  ...,  0.0141,  0.0164, -0.0147],
        [-0.0012, -0.0015, -0.0108,  ...,  0.0174, -0.0097, -0.0093]],
       requires_grad=True)

In [57]:
import os
#os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:2' # do this before importing pytorch
import torch
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(), lr=3e-4, betas=(0.9, 0.95), weight_decay=0.1)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=100,
                                            num_training_steps=num_epoch * len(dataloader))

In [58]:
num_epoch * len(dataloader)

1861028

In [59]:
device = torch.device(str("cuda:0") if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [60]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [61]:
for epoch in range(num_epoch):
    print('epoch number is ', str(epoch))
    model.train()
    for step, batch in enumerate(dataloader):
        if step == 3:
            break
        optimizer.zero_grad()
        for k, v in batch.items():
            batch = {k: v.cuda() for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        scheduler.step()
        print('training loss:')
        print(loss)

    model.eval()
    with torch.no_grad():
        for step, batch_valid in enumerate(valid_dataloader):
            if step == 3:
                break
            for k_, v_ in batch_valid.items():
                batch_valid = {k_: v_.cuda() for k_, v_ in batch_valid.items()}
            outputs_valid = model(**batch_valid)
            loss_valid = outputs_valid[0].item()
            print('validation loss is', loss_valid)

epoch number is  0
training loss:
tensor(10.4766, device='cuda:0', grad_fn=<NllLossBackward0>)
training loss:
tensor(10.5344, device='cuda:0', grad_fn=<NllLossBackward0>)
training loss:
tensor(10.4629, device='cuda:0', grad_fn=<NllLossBackward0>)
validation loss is 11.01328182220459
epoch number is  1
training loss:
tensor(10.2157, device='cuda:0', grad_fn=<NllLossBackward0>)
training loss:
tensor(10.1563, device='cuda:0', grad_fn=<NllLossBackward0>)
training loss:
tensor(10.0343, device='cuda:0', grad_fn=<NllLossBackward0>)
validation loss is 10.708047866821289
epoch number is  2
training loss:
tensor(9.7984, device='cuda:0', grad_fn=<NllLossBackward0>)
training loss:
tensor(9.7844, device='cuda:0', grad_fn=<NllLossBackward0>)
training loss:
tensor(9.7133, device='cuda:0', grad_fn=<NllLossBackward0>)
validation loss is 10.701873779296875
epoch number is  3
training loss:
tensor(9.4875, device='cuda:0', grad_fn=<NllLossBackward0>)
training loss:
tensor(9.5191, device='cuda:0', grad_fn=

In [36]:
loss_valid

10.730388641357422

In [45]:
loss

tensor(9.7198, device='cuda:0', grad_fn=<NllLossBackward0>)

In [115]:
logit = outputs.logits
logit

tensor([[[-0.7465, -1.2179, -0.6294,  ...,  0.5506,  0.2126, -0.6999],
         [-0.3763, -1.2060, -0.7949,  ...,  0.4821,  0.1066, -0.6537],
         [-0.4480, -1.2591, -0.6291,  ...,  0.4949,  0.2238, -0.6204],
         ...,
         [ 0.0123, -0.9344, -0.5872,  ...,  0.4297,  0.5233, -0.4541],
         [-0.2127, -0.9061, -0.6173,  ...,  0.3971,  0.4578, -0.4449],
         [-0.0504, -1.0030, -0.6453,  ...,  0.3347,  0.3408, -0.4290]]],
       device='cuda:0', grad_fn=<UnsafeViewBackward0>)

In [116]:
import numpy as np
np.shape(logit)

torch.Size([1, 1024, 32000])

In [117]:
predicted_token_ids = torch.argmax(logit, dim=-1)
print(np.shape(predicted_token_ids))
predicted_token_ids

torch.Size([1, 1024])


tensor([[29906, 29906, 29906,  ...,    13,    13,    13]], device='cuda:0')

In [118]:
predicted_token_ids[0]

tensor([29906, 29906, 29906,  ...,    13,    13,    13], device='cuda:0')

In [120]:
predicted_text = tokenizer.decode(predicted_token_ids[0])  # Assuming batch size is 1
print(predicted_text)

222222222222222222222222222222222222222222222

2


22
2


















































































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [126]:
logit = outputs_valid.logits
predicted_token_ids = torch.argmax(logit, dim=-1)
predicted_text = tokenizer.decode(predicted_token_ids[0])  # Assuming batch size is 1
print(predicted_text)

222






































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [121]:
tokenizer.decode(batch['input_ids'][0])

'<s>\\section{Introduction}\n\\label{sec:intro}\n\n\\emph{Gender diversity}, or more often its lack thereof, among participants to\nsoftware development activities has been thoroughly studied in recent years. In\nparticular, the presence of, effects of, and countermeasures for \\emph{gender\n  bias} in Free/Open Source Software (FOSS) have received a lot of attention\nover the past decade~\\cite{david2008fossdevs, qiu2010kdewomen,\n  nafus2012patches, kuechler2012genderfoss, vasilescu2014gender,\n  oneil2016debiansurvey, robles2016womeninfoss, terrell2017gender,\n  zacchiroli2021gender}.  \\emph{Geographic diversity} is on the other hand the\nkind of diversity that stems from participants in some global activity coming\nfrom different world regions and cultures.\n\nGeographic diversity in FOSS has received relatively little attention in scholarly\nworks. In particular, while seminal survey-based and\npoint-in-time medium-scale studies of the geographic origins of FOSS\ncontributors exi

In [103]:
batch['input_ids']

tensor([[   1, 4001,  372,  ...,    0,    0,    0]], device='cuda:0')

In [63]:
batch

{'input_ids': tensor([[    1,   320,  2042,  ...,   424, 23460, 29889]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'),
 'labels': tensor([[    1,   320,  2042,  ...,   424, 23460, 29889]], device='cuda:0')}

In [199]:
from tqdm import tqdm
for epoch in range(start_epoch, num_epoch):
    model.train()
    with tqdm(enumerate(dataloader),
              desc=f'Epoch {epoch}',
              total=num_steps_per_epoch,
              initial=start_step) as pbar:
        for step, batch in pbar:
            optimizer.zero_grad()
            for k, v in batch.items():
                batch = {k: v.cuda() for k, v in batch.items()}
                break
            outputs = model(**batch)
            loss = outputs[0]
            loss.backward()
            optimizer.step()
            scheduler.step()
            break

    model.eval()
    with tqdm(enumerate(valid_dataloader),
              desc=f'Epoch {epoch}',
              initial=start_step) as pbar:
        with torch.no_grad():
            for step, batch_valid in enumerate(valid_dataloader):
                for k_, v_ in batch_valid.items():
                    batch_valid = {k_: v_.cuda() for k_, v_ in batch_valid.items()}
                outputs_valid = model(**batch_valid)
                loss_valid = outputs_valid[0].item()

Epoch 0:   0%|          | 0/930514 [00:00<?, ?it/s]
Epoch 0: 0it [00:00, ?it/s]


In [200]:
loss

tensor(10.5123, device='cuda:0', grad_fn=<NllLossBackward0>)

In [201]:
loss_valid

11.107804298400879

In [21]:
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler
for epoch in range(start_epoch, num_epoch):
    model.train()
    with tqdm(enumerate(dataloader),
              desc=f'Epoch {epoch}',
              total=num_steps_per_epoch,
              initial=start_step) as pbar:
        for step, batch in pbar:
            optimizer.zero_grad()
            with autocast(): # handle mixed precisions
                for k, v in batch.items():
                    batch = {k: v.cuda()}
                    break
                #batch = {k: v.cuda() for k, v in batch.items()}
                outputs = model(**batch)
                loss = outputs[0]
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) # gradient clipping to prevent overflows
            optimizer.step()
            scheduler.step()
            break

Epoch 0:   0%|          | 0/465257 [00:00<?, ?it/s]


RuntimeError: value cannot be converted to type at::Half without overflow

In [24]:
outputs

NameError: name 'outputs' is not defined

In [131]:
np.shape(v)

torch.Size([2, 1024])

In [120]:
tokenizer.pad_token_id

0

In [124]:
text1 = loaded_data_lama['train']['text'][0]

In [127]:
text1

"\\section{Introduction}\n\\label{sec:intro}\n\n\\emph{Gender diversity}, or more often its lack thereof, among participants to\nsoftware development activities has been thoroughly studied in recent years. In\nparticular, the presence of, effects of, and countermeasures for \\emph{gender\n  bias} in Free/Open Source Software (FOSS) have received a lot of attention\nover the past decade~\\cite{david2008fossdevs, qiu2010kdewomen,\n  nafus2012patches, kuechler2012genderfoss, vasilescu2014gender,\n  oneil2016debiansurvey, robles2016womeninfoss, terrell2017gender,\n  zacchiroli2021gender}.  \\emph{Geographic diversity} is on the other hand the\nkind of diversity that stems from participants in some global activity coming\nfrom different world regions and cultures.\n\nGeographic diversity in FOSS has received relatively little attention in scholarly\nworks. In particular, while seminal survey-based and\npoint-in-time medium-scale studies of the geographic origins of FOSS\ncontributors exist~

In [126]:
tokenizer(text1)

{'input_ids': [1, 320, 2042, 29912, 25898, 29913, 13, 29905, 1643, 29912, 3471, 29901, 23333, 29913, 13, 13, 29905, 7278, 29912, 29954, 1581, 6894, 537, 1118, 470, 901, 4049, 967, 10225, 727, 974, 29892, 4249, 27138, 304, 13, 20415, 5849, 14188, 756, 1063, 26606, 12399, 297, 7786, 2440, 29889, 512, 13, 1595, 16311, 29892, 278, 10122, 310, 29892, 9545, 310, 29892, 322, 6795, 1004, 25414, 363, 320, 7278, 29912, 26098, 13, 29871, 24003, 29913, 297, 12362, 29914, 6585, 7562, 18540, 313, 5800, 1799, 29897, 505, 4520, 263, 3287, 310, 8570, 13, 957, 278, 4940, 316, 6332, 2651, 2036, 29912, 29881, 16093, 29906, 29900, 29900, 29947, 29888, 2209, 3359, 29879, 29892, 3855, 5871, 29906, 29900, 29896, 29900, 29895, 311, 29893, 2770, 29892, 13, 29871, 1055, 29888, 375, 29906, 29900, 29896, 29906, 5041, 267, 29892, 413, 434, 305, 1358, 29906, 29900, 29896, 29906, 26098, 29888, 2209, 29892, 19723, 5475, 4979, 29906, 29900, 29896, 29946, 26098, 29892, 13, 29871, 697, 309, 29906, 29900, 29896, 29953, 31

In [130]:
tokenizer(text1, return_tensors="pt", padding='max_length', truncation=True, max_length=1024)

{'input_ids': tensor([[    1,   320,  2042,  ...,   424, 23460, 29889]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}

In [44]:
for epoch in range(start_epoch, num_epochs):
    with tqdm(enumerate(dataloader),
                desc=f'Epoch {epoch}',
                disable=not coordinator.is_master(),
                total=num_steps_per_epoch,
                initial=start_step) as pbar:
        for step, batch in pbar:
            batch = {k: v.cuda() for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs[0]
            booster.backward(loss, optimizer)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

            all_reduce_mean(loss)
            pbar.set_postfix({'loss': loss.item()})
            if coordinator.is_master():
                writer.add_scalar('loss', loss.item(), epoch * num_steps_per_epoch + step)

            if args.save_interval > 0 and (step + 1) % args.save_interval == 0:
                coordinator.print_on_master(f'Saving checkpoint')
                save(booster, model, optimizer, lr_scheduler, epoch, step + 1, args.batch_size, coordinator,
                        args.save_dir)
                coordinator.print_on_master(f'Saved checkpoint at epoch {epoch} step {step + 1}')
    # the continue epochs are not resumed, so we need to reset the sampler start index and start step
    dataloader.sampler.set_start_index(0)
    start_step = 0

coordinator.print_on_master(f'Max CUDA memory usage: {torch.cuda.max_memory_allocated()/1024**2:.2f} MB')

In [5]:
path = '../../protgpt2/'

tokenizer = Tokenizer.from_file(path+'tokenizer.json')
# Load special tokens from JSON
with open(path + 'special_tokens_map.json', 'r') as f:
    special_tokens_map = json.load(f)

# Extract actual tokens from the loaded map
special_tokens_list = [
    AddedToken(special_tokens_map.get('bos_token', {}).get('content'), single_word=False),
    AddedToken(special_tokens_map.get('eos_token', {}).get('content'), single_word=False),
    AddedToken(special_tokens_map.get('unk_token', {}).get('content'), single_word=False)
]

# Remove any tokens that are None or empty
filtered_tokens = [token for token in special_tokens_list if token.content]

tokenizer.add_special_tokens(filtered_tokens)

0

In [6]:
bos_token_id  = tokenizer.token_to_id("<|endoftext|>")
print(bos_token_id)

0


In [7]:
tokenizer.get_vocab('padding')

TypeError: argument 'with_added_tokens': 'str' object cannot be converted to 'PyBool'

In [61]:
encoding = tokenizer.encode("SLGPVADL").ids

In [62]:
tokenizer.token_to_id("<BOS>")

50257

In [63]:
[tokenizer.token_to_id("<BOS>")] + encoding

[50257, 556, 48, 2459]

In [58]:
print("Encoded string: {}".format(encoding.tokens))

decoded = tokenizer.decode(encoding.ids)
print("Decoded string: {}".format(decoded))

Encoded string: ['SLG', 'P', 'VADL']
Decoded string: SLGPVADL


In [18]:
encoding.tokens

['SLG', 'P', 'VADL']

In [19]:
encoding.ids

[556, 48, 2459]

In [20]:
from sentencepiece import SentencePieceProcessor
sp_model = SentencePieceProcessor(model_file=path+'tokenizer.json')

RuntimeError: Internal: /sentencepiece/python/bundled/sentencepiece/src/sentencepiece_processor.cc(848) [model_proto->ParseFromArray(serialized.data(), serialized.size())] 